# 2.3 Data Access and Basic Processing Pangeo Apply

<center><img src="https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/icons/cnc_3icons_process_circle.svg"
     alt="Cubes & Clouds logo"
     style="float: center; margin-right: 10px; margin-left: 10px; max-height: 250px;" /></center>

<img src="https://raw.githubusercontent.com/pangeo-data/pangeo.io/refs/heads/main/public/Pangeo-assets/pangeo_logo.png"
     alt="Pangeo logo"
     style="float: center; margin-right: 10px; max-height: 80px;"/>

## Apply Operator with Pangeo ecosystem

The apply operator applies a process to each value in the data cube (i.e. a local operation).

Let's start again with the same sample data from the Sentinel-2 STAC Collection, applying the filters directly in the `stackstac.stack` call, to reduce the amount of data.

In [ ]:
import pystac_client
import stackstac
import xarray as xr

In [ ]:
#                  West,     South,     East,      North
spatial_extent = [11.259613, 46.461019, 11.406212, 46.522237]
temporal_extent = ['2022-07-10T00:00:00Z','2022-07-13T00:00:00Z']
bands = ["red","green","blue"]

In [ ]:
URL = "https://earth-search.aws.element84.com/v1"
catalog = pystac_client.Client.open(URL)
s2_items = catalog.search(
    bbox=spatial_extent,
    datetime=temporal_extent,
    query=["eo:cloud_cover<50"],
    collections=["sentinel-2-l2a"]
).item_collection()

s2_cube = stackstac.stack(s2_items,
                     bounds_latlon=spatial_extent,
                     assets=bands
)
s2_cube

Visualize the RGB bands of our sample dataset:

In [ ]:
s2_cube.isel(time=0).plot.imshow()

### Apply an arithmetic formula

We would like to improve the previous visualization, rescaling all the pixels between 0 and 1.

We can use `apply` with an ad-hoc `rescale` function.

In [ ]:
input_min = -0.1
input_max = 0.2
output_min = 0
output_max = 1


def rescale(arr):
    norm_arr = arr.clip(min=input_min, max=input_max)
    norm_arr = ((norm_arr - input_min) / (input_max - input_min)) * (output_max - output_min) + output_min
    return norm_arr

scaled_data = s2_cube.to_dataset().apply(rescale)

## Visualise the scaled image

Visualize the result and see how `apply` scaled the data resulting in a more meaningful visualization:

In [ ]:
scaled_data.to_dataarray().squeeze().plot.imshow()